In [1]:
import os
from typing import List

import fitz

from rich import print
from rich.pretty import pprint

# DATA PREPARATION

## SPLIT THE MAIN PDF BY EACH SECTION

In [ ]:
def partition_pdf(page_numbers: List[int], new_pdf_name: str, pdf_path: str = "data/raw/Master_Thesis_Aidana_Rakhatbekova.pdf") -> None:
    """
    Partitions a PDF by selecting specific pages and saves the result to a new PDF file.

    Parameters:
        page_numbers (List[int]): A list of page numbers to select from the original PDF.
        new_pdf_name (str): The file path where the new PDF will be saved.
        pdf_path (str, optional): The file path of the original PDF. Defaults to "data/raw/Master_Thesis_Aidana_Rakhatbekova.pdf".

    Returns:
        None
    """
    doc = fitz.open(pdf_path)
    doc.select(page_numbers)
    doc.save(new_pdf_name)

In [ ]:
partition_pdf([1, 2, 3, 4, 5], "data/processed/Introduction.pdf")

In [ ]:
partition_pdf([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], "data/processed/State_of_the_Art.pdf")

In [ ]:
partition_pdf([18, 19, 20, 21], "data/processed/Theoretical_Framework.pdf")

In [ ]:
partition_pdf([21, 22, 23, 24, 25, 26], "data/processed/Methodology.pdf")

In [ ]:
partition_pdf([27, 28, 29, 30, 
               31, 32, 33, 34, 35, 
               36, 37, 38, 39, 40,
               41, 42, 43, 44, 45, 
               46, 47, 48, 49, 50, 
               51, 52, 53, 54, 55, 
               56, 57, 58, 59], "data/processed/Analysis_and_Presentation_of_Findings.pdf")

In [ ]:
partition_pdf([59, 60, 61, 62, 63], "data/processed/Conclusion.pdf")

In [ ]:
partition_pdf([64, 65, 66, 67, 68, 69, 70], "data/processed/References.pdf")

In [ ]:
partition_pdf([71, 72], "data/processed/Appendix_A.pdf")

In [ ]:
partition_pdf([75], "data/processed/Appendix_B.pdf")

## 

## LLamaParse to Parse PDF to Markdown

In [2]:
from constants import (AZURE_OPENAI_ENDPOINT,
                       AZURE_OPENAI_API_KEY,
                       AZURE_OPENAI_API_VERSION,
                       DB_CONNECTION,
                       EMBEDDING_MODEL,
                       LLM,
                       LLAMAPARSE_API_KEY,
                       LANGCHAIN_URL,
                       LANGCHAIN_API_KEY,
                       LANGCHAIN_PROJECT    
                       )

In [3]:
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_URL
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

parser = LlamaParse(
api_key=LLAMAPARSE_API_KEY,
result_type="markdown",  # "markdown" and "text" are available
verbose=True,
language="en")

file_extractor = {".pdf": parser}

In [ ]:
def extract_markdown_from_pdf(api_key: str = LLAMAPARSE_API_KEY, folder_path: str = "data/processed"):
    from llama_parse import LlamaParse
    from llama_index.core import SimpleDirectoryReader

    parser = LlamaParse(
    api_key=LLAMAPARSE_API_KEY,
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True,
    language="en")

    file_extractor = {".pdf": parser}
    
    documents = SimpleDirectoryReader("./data", file_extractor=file_extractor).load_data()

    print("Total Number of Documents parsed ->", len(documents))

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser

markdown_parser = MarkdownNodeParser()
nodes = markdown_parser.get_nodes_from_documents(documents, include_metadata=True, include_prev_next_rel=True)

In [ ]:
chapters = ["INTRODUCTION",
            "STATE OF THE ART",
            "THEORETICAL FRAMEWORK",
            "METHODOLOGY",
            "ANALYSIS AND PRESENTATION OF FINDINGS",
            "CONCLUSION",
            "APPENDIX A",
            "APPENDIX B"]

"A futuristic, beautiful brain with intricate and glowing neural pathways, set against a sleek, high-tech background. The brain should look advanced and ethereal, with elements of cyberpunk and bioluminescence. Below the brain, the words 'The Decolonized Mind' are written in elegant, modern typography, seamlessly integrated into the design."

In [5]:
from langchain_openai import AzureChatOpenAI

In [6]:
llm = AzureChatOpenAI(            
        temperature = 0.2,
        openai_api_version = AZURE_OPENAI_API_VERSION,
        azure_endpoint = AZURE_OPENAI_ENDPOINT,
        openai_api_key = AZURE_OPENAI_API_KEY,         
        model_name = LLM
        )

In [7]:
from langchain_core.messages import HumanMessage

In [8]:
message = HumanMessage(
    content="Translate this sentence from English to Hindi. I love programming."
)

In [9]:
llm.invoke([message])

AIMessage(content='मुझे प्रोग्रामिंग करना बहुत पसंद है।', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 19, 'total_tokens': 55}, 'model_name': 'gpt-4-32k:swedencentral', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-9c1af7b5-963e-44af-9664-6abbfc444336-0', usage_metadata={'input_tokens': 19, 'output_tokens': 36, 'total_tokens': 55})

In [17]:
from openai import AzureOpenAI

ai_client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
)

In [18]:
import time

In [19]:
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [22]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_embeddings(text: str):
    '''
    Generate embeddings from string of text using the deployed Azure OpenAI API embeddings model.
    This will be used to vectorize document data and incoming user messages for a similarity search with
    the vector index.
    '''
    response = ai_client.embeddings.create(input=text, model=EMBEDDING_MODEL)
    embeddings = response.data[0].embedding
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI
    return embeddings

In [24]:
# demonstrate embeddings generation using a test string
# test = "hello, world"
# print(generate_embeddings(test))

In [11]:
from langchain_community.vectorstores import AzureCosmosDBVectorSearch

In [ ]:
vector_store = AzureCosmosDBVectorSearch.from_connection_string(
    connection_string = DB_CONNECTION,
    namespace = "test",
    embedding = embedding_model,
    index_name = "test_index",
    embedding_key = "content_vector"

)

In [12]:
import pymongo

In [13]:
client = pymongo.MongoClient(DB_CONNECTION)

/tmp/ipykernel_2531/1043822493.py:1: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = pymongo.MongoClient(DB_CONNECTION)


In [14]:
db = client.thesis_vectordb

In [15]:
db

Database(MongoClient(host=['c.pdf-ag-cosmos-mongodb.mongocluster.cosmos.azure.com:10260'], document_class=dict, tz_aware=False, connect=True, tls=True, authmechanism='SCRAM-SHA-256', retrywrites=False, maxidletimems=120000), 'thesis_vectordb')

In [16]:
collection = db.introduction